In [1]:
import pandas as pd
import numpy as np
import re
import math
import tabula
import camelot
try:
    from cStringIO import StringIO
except ImportError:
    from io import StringIO

In [32]:
f = r'C:\Users\MudraCircle\Desktop\bks_raw\Parsing_testing\KOTAK\files\type3\kotak01.pdf'
tables=tabula.read_pdf(f,
                          lattice=True,
                          pages="all",
                          silent=True,
                          multiple_tables=True,
                          pandas_options={'header':None})

In [33]:
df = pd.DataFrame()
df = pd.concat([c for c in tables]).drop_duplicates()
df.to_csv("check.csv", index=0)

In [34]:
df = df.replace(r'^\s*$', np.nan, regex=True)
df = df[ df.isnull().sum(axis=1) < df.shape[1]-2].reset_index(drop=True)
df.head()

,0,1,2,3,4,5,6,7
0,Deposit Accounts-INR,NaN,AssetClass,NaN,Total Balance,NaN,NaN,NaN
1,NaN,Date,Narration,Chq/Ref No.,Withdrawal (Dr),NaN,Deposit (Cr),Balance
2,01-Dec-19,OPENING BALANCE ...,NaN,NaN,"5,699.62","5,699.62 (Cr)",NaN,NaN
3,02-Dec-19,MB:ac emi,000145922810,"5,000.00",NaN,699.62 (Cr),NaN,NaN
4,02-Dec-19,Received from Payt XX1136 IMPS REF\r933605909687,IMPS-933605460712,NaN,"3,700.00","4,399.62 (Cr)",NaN,NaN


In [35]:
try:
    idx=[c for c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('balance').any(),axis=1)==True].index if c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('date').any(),axis=1)==True].index][0]
    df.columns=df.iloc[idx];df=df.iloc[idx+1:,:]; df.reset_index(drop=True, inplace=True)
except:
    try:
        idx=[c for c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('balance').any(),axis=1)==True].index if c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('transaction').any(),axis=1)==True].index][0]
        df.columns=df.iloc[idx];df=df.iloc[idx+1:,:]; df.reset_index(drop=True, inplace=True)
    except:
        try:
            idx=[ c for c in df[df.apply(lambda row: row.astype(str).str.lower().str.contains('date').any(), axis=1) ==True].index if c in df[df.apply(lambda row: row.astype(str).str.lower().str.contains('remarks').any(), axis=1) ==True].index ][0]
            df.columns=df.iloc[idx];df=df.iloc[idx+1:,:] ; df.reset_index(drop=True,inplace=True)
        except:
            try:
                idx=[ c for c in df[df.apply(lambda row: row.astype(str).str.lower().str.contains('date').any(), axis=1) ==True].index if c in df[df.apply(lambda row: row.astype(str).str.lower().str.contains('narration').any(), axis=1) ==True].index ][0]
                df.columns=df.iloc[idx];df=df.iloc[idx+1:,:] ; df.reset_index(drop=True,inplace=True)
            except:
                print("\nKOTAK Column Headers Missing"); pass
            
try:
    idx2 = [c for c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('statement summary').any(),axis=1)==True].index][0]    
    df.drop(df.index[idx2:], inplace=True)
except:pass

In [36]:
df.head()

1,NaN,Date,Narration,Chq/Ref No.,Withdrawal (Dr),NaN,Deposit (Cr),Balance
0,01-Dec-19,OPENING BALANCE ...,NaN,NaN,"5,699.62","5,699.62 (Cr)",NaN,NaN
1,02-Dec-19,MB:ac emi,000145922810,"5,000.00",NaN,699.62 (Cr),NaN,NaN
2,02-Dec-19,Received from Payt XX1136 IMPS REF\r933605909687,IMPS-933605460712,NaN,"3,700.00","4,399.62 (Cr)",NaN,NaN
3,02-Dec-19,KOTAKPAYOUT-057035500008-011219,0000000000008275535,NaN,"7,222.47","11,622.09 (Cr)",NaN,NaN
4,02-Dec-19,KOTAKPAYOUT-057035500008-021219,0000000000008286124,NaN,"3,769.73","15,391.82 (Cr)",NaN,NaN


In [37]:
def isnan(value):
    try:
        return math.isnan(float(value))
    except:
        return False


list1 = list(df.columns)
list2 = []
for i in list1:
    if isnan(i) == False:
        list2.append(i)
df = df.iloc[:,:len(list2)]        
df.columns = list2

In [38]:
df.to_csv("check.csv", index=0)

In [39]:
def abc(a):
    if type(a) == str:
        if len(a.split(' '))==2:
            z=a.split(' ')[1]
        else:
            z=a.split(' ')[0]
    else:
        z=a
    return z

try:
    dat=[c for c in df.columns if "TRANSACTION DATE" in str(c).upper() ][0]
except:
    try:
        dat=[c for c in df.columns if "TXN DATE" in str(c).upper() ][0]
    except:
        try:
            dat=[c for c in df.columns if "DATE" in str(c).upper() ][0]
        except:pass
        
try:
    chq=[c for c in df.columns if "CHQ" in str(c).upper() ][0]
except:
    try:
        chq=[c for c in df.columns if "CHEQUE" in str(c).upper() ][0]
    except:pass
    
try:
    narr=[c for c in df.columns if "REMARKS" in str(c).upper() ][0]
except:
    try:
        narr=[c for c in df.columns if "PARTICULARS" in str(c).upper() ][0]
    except:
        try:
            narr=[c for c in df.columns if "DESCRIPTION" in str(c).upper() ][0]
        except:
            try:
                narr=[c for c in df.columns if "DETAILS" in str(c).upper() ][0]
            except:
                try:
                    narr=[c for c in df.columns if "NARRATION" in str(c).upper() ][0]
                except:pass
            
try:
    bal=[c for c in df.columns if "BALANCE" in str(c).upper() ][0]
    df["bal"]=df[bal].apply( lambda x: abc(x) )
except: print("\nBalance columns missing") 

try:
    wdl=[c for c in df.columns if "WITHDRAW" in str(c).upper() ][0]
except:
    try:
        wdl=[c for c in df.columns if "DEBIT" in str(c).upper() ][0]
    except: pass 

try:
    dep=[c for c in df.columns if "DEPOSIT" in str(c).upper() ][0]
except:
    try:
        dep=[c for c in df.columns if "CREDIT" in str(c).upper() ][0]
    except: pass

print(dat,chq,narr,bal,wdl,dep)

Date Chq/Ref No. Narration Balance Withdrawal (Dr) Deposit (Cr)


In [40]:
df.head()

,Date,Narration,Chq/Ref No.,Withdrawal (Dr),Deposit (Cr),Balance,bal
0,01-Dec-19,OPENING BALANCE ...,NaN,NaN,"5,699.62","5,699.62 (Cr)",(Cr)
1,02-Dec-19,MB:ac emi,000145922810,"5,000.00",NaN,699.62 (Cr),(Cr)
2,02-Dec-19,Received from Payt XX1136 IMPS REF\r933605909687,IMPS-933605460712,NaN,"3,700.00","4,399.62 (Cr)",(Cr)
3,02-Dec-19,KOTAKPAYOUT-057035500008-011219,0000000000008275535,NaN,"7,222.47","11,622.09 (Cr)",(Cr)
4,02-Dec-19,KOTAKPAYOUT-057035500008-021219,0000000000008286124,NaN,"3,769.73","15,391.82 (Cr)",(Cr)


In [41]:
df[dep]=df[dep].apply( lambda x: x.split(' ')[0] if type(x) == str else x )
df[wdl]=df[wdl].apply( lambda x: x.split(' ')[0] if type(x) == str else x )
df[wdl]=df[wdl].astype(str).apply(lambda x: str(x).replace("\r","").replace(",","").replace("(Cr)","").replace("(Dr)","")).astype(float) *-1
df[dep]=df[dep].astype(str).apply(lambda x: str(x).replace("\r","").replace(",","").replace("(Cr)","").replace("(Dr)","")).astype(float)
df[bal]=df[bal].astype(str).apply(lambda x:str(x).replace("\r","").replace(",","").replace("(Cr)","").replace("(Dr)","")).astype(float)

try:
    for i in df.index:
        if 'Cr' in df["bal"][i]:
            df[bal][i] = df[bal][i]
        elif 'Dr' in df["bal"][i]:
            df[bal][i] = df[bal][i]*-1
        else:
            pass
except:pass

c:\users\mudracircle\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [42]:
df.head()

,Date,Narration,Chq/Ref No.,Withdrawal (Dr),Deposit (Cr),Balance,bal
0,01-Dec-19,OPENING BALANCE ...,NaN,NaN,5699.62,5699.62,(Cr)
1,02-Dec-19,MB:ac emi,000145922810,-5000.0,NaN,699.62,(Cr)
2,02-Dec-19,Received from Payt XX1136 IMPS REF\r933605909687,IMPS-933605460712,NaN,3700.00,4399.62,(Cr)
3,02-Dec-19,KOTAKPAYOUT-057035500008-011219,0000000000008275535,NaN,7222.47,11622.09,(Cr)
4,02-Dec-19,KOTAKPAYOUT-057035500008-021219,0000000000008286124,NaN,3769.73,15391.82,(Cr)


In [43]:
df = df[[dat,chq,narr,wdl,dep,bal]]
df.columns = ["Xns Date","Cheque No","Narration","Debits","Credits","Balance"]

In [44]:
df.to_csv("check.csv", index=0)